In [173]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression #로지스틱 회귀
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier #랜덤포레스트
import tensorflow as tf
from tensorflow.keras import layers
from catboost import Pool,CatBoostClassifier
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 

In [174]:
tmp_train=pd.read_csv("train.csv").set_index(keys='id')
tmp_test=pd.read_csv("test.csv").set_index(keys='id')
d_code=pd.read_csv("속성_D_코드.csv").set_index(keys='속성 D 코드')
h_code=pd.read_csv("속성_H_코드.csv").set_index(keys='속성 H 코드')
l_code=pd.read_csv("속성_L_코드.csv").set_index(keys='속성 L 코드')

In [175]:
tmp_train.isna().sum()

d_l_match_yn              0
d_m_match_yn              0
d_s_match_yn              0
h_l_match_yn              0
h_m_match_yn              0
h_s_match_yn              0
person_attribute_a        0
person_attribute_a_1      0
person_attribute_b        0
person_prefer_c           0
person_prefer_d_1         0
person_prefer_d_2         0
person_prefer_d_3         0
person_prefer_e           0
person_prefer_f           0
person_prefer_g           0
person_prefer_h_1         0
person_prefer_h_2         0
person_prefer_h_3         0
contents_attribute_i      0
contents_attribute_a      0
contents_attribute_j_1    0
contents_attribute_j      0
contents_attribute_c      0
contents_attribute_k      0
contents_attribute_l      0
contents_attribute_d      0
contents_attribute_m      0
contents_attribute_e      0
contents_attribute_h      0
person_rn                 0
contents_rn               0
contents_open_dt          0
target                    0
dtype: int64

In [176]:
tmp_test.isna().sum()

d_l_match_yn              0
d_m_match_yn              0
d_s_match_yn              0
h_l_match_yn              0
h_m_match_yn              0
h_s_match_yn              0
person_attribute_a        0
person_attribute_a_1      0
person_attribute_b        0
person_prefer_c           0
person_prefer_d_1         0
person_prefer_d_2         0
person_prefer_d_3         0
person_prefer_e           0
person_prefer_f           0
person_prefer_g           0
person_prefer_h_1         0
person_prefer_h_2         0
person_prefer_h_3         0
contents_attribute_i      0
contents_attribute_a      0
contents_attribute_j_1    0
contents_attribute_j      0
contents_attribute_c      0
contents_attribute_k      0
contents_attribute_l      0
contents_attribute_d      0
contents_attribute_m      0
contents_attribute_e      0
contents_attribute_h      0
person_rn                 0
contents_rn               0
contents_open_dt          0
dtype: int64

In [177]:
def createDF(main_df,d_code=d_code,h_code=h_code,l_code=l_code):
    main_df=main_df.copy()
    main_df["person_prefer_d_1_n"]=main_df["person_prefer_d_1"].apply(lambda x: d_code.loc[x,"속성 D 세분류코드"])
    main_df["person_prefer_d_1_s"]=main_df["person_prefer_d_1"].apply(lambda x: d_code.loc[x,"속성 D 소분류코드"])
    main_df["person_prefer_d_1_m"]=main_df["person_prefer_d_1"].apply(lambda x: d_code.loc[x,"속성 D 중분류코드"])
    main_df["person_prefer_d_1_l"]=main_df["person_prefer_d_1"].apply(lambda x: d_code.loc[x,"속성 D 대분류코드"])
    
    main_df["person_prefer_d_2_n"]=main_df["person_prefer_d_2"].apply(lambda x: d_code.loc[x,"속성 D 세분류코드"])
    main_df["person_prefer_d_2_s"]=main_df["person_prefer_d_2"].apply(lambda x: d_code.loc[x,"속성 D 소분류코드"])
    main_df["person_prefer_d_2_m"]=main_df["person_prefer_d_2"].apply(lambda x: d_code.loc[x,"속성 D 중분류코드"])
    main_df["person_prefer_d_2_l"]=main_df["person_prefer_d_2"].apply(lambda x: d_code.loc[x,"속성 D 대분류코드"])
    
    main_df["person_prefer_d_3_n"]=main_df["person_prefer_d_3"].apply(lambda x: d_code.loc[x,"속성 D 세분류코드"])
    main_df["person_prefer_d_3_s"]=main_df["person_prefer_d_3"].apply(lambda x: d_code.loc[x,"속성 D 소분류코드"])
    main_df["person_prefer_d_3_m"]=main_df["person_prefer_d_3"].apply(lambda x: d_code.loc[x,"속성 D 중분류코드"])
    main_df["person_prefer_d_3_l"]=main_df["person_prefer_d_3"].apply(lambda x: d_code.loc[x,"속성 D 대분류코드"])
    
    main_df['person_prefer_h_1_m']=main_df['person_prefer_h_1'].apply(lambda x: h_code.loc[x,"속성 H 중분류코드"])
    main_df['person_prefer_h_1_l']=main_df['person_prefer_h_1'].apply(lambda x: h_code.loc[x,"속성 H 대분류코드"])
    
    main_df['person_prefer_h_2_m']=main_df['person_prefer_h_2'].apply(lambda x: h_code.loc[x,"속성 H 중분류코드"])
    main_df['person_prefer_h_2_l']=main_df['person_prefer_h_2'].apply(lambda x: h_code.loc[x,"속성 H 대분류코드"])
    
    main_df['person_prefer_h_3_m']=main_df['person_prefer_h_3'].apply(lambda x: h_code.loc[x,"속성 H 중분류코드"])
    main_df['person_prefer_h_3_l']=main_df['person_prefer_h_3'].apply(lambda x: h_code.loc[x,"속성 H 대분류코드"])
    
    main_df['contents_attribute_l_n']=main_df['contents_attribute_l'].apply(lambda x: l_code.loc[x,"속성 L 세분류코드"])
    main_df['contents_attribute_l_s']=main_df['contents_attribute_l'].apply(lambda x: l_code.loc[x,"속성 L 소분류코드"])
    main_df['contents_attribute_l_m']=main_df['contents_attribute_l'].apply(lambda x: l_code.loc[x,"속성 L 중분류코드"])
    main_df['contents_attribute_l_l']=main_df['contents_attribute_l'].apply(lambda x: l_code.loc[x,"속성 L 대분류코드"])
    
    main_df['contents_attribute_d_n']=main_df['contents_attribute_d'].apply(lambda x: d_code.loc[x,"속성 D 세분류코드"])
    main_df['contents_attribute_d_s']=main_df['contents_attribute_d'].apply(lambda x: d_code.loc[x,"속성 D 소분류코드"])
    main_df['contents_attribute_d_m']=main_df['contents_attribute_d'].apply(lambda x: d_code.loc[x,"속성 D 중분류코드"])
    main_df['contents_attribute_d_l']=main_df['contents_attribute_d'].apply(lambda x: d_code.loc[x,"속성 D 대분류코드"])

    main_df['contents_attribute_h_m']=main_df['contents_attribute_h'].apply(lambda x: h_code.loc[x,"속성 H 중분류코드"])
    main_df['contents_attribute_h_l']=main_df['contents_attribute_h'].apply(lambda x: h_code.loc[x,"속성 H 대분류코드"])
    
    return main_df
    

In [178]:
def preprocessing(df:pd.DataFrame)->pd.DataFrame:
    print('--------start---------')
    df=df.copy()
    df.drop(['person_prefer_d_1','person_prefer_d_2','person_prefer_d_3','person_prefer_h_1',
             'person_prefer_h_2','person_prefer_h_3','contents_attribute_l','contents_attribute_d',
             'contents_attribute_h','person_prefer_f','person_prefer_g','contents_rn'],axis=1,inplace=True)
    print('---------mid_1----------')
    for column in df.keys():
        if column=='contents_open_dt':
            df[column]=df[column].astype('datetime64').dt.dayofweek #0~6(0부터 월요일~)
        if df[column].dtypes!='datetime64':
            df[column]=df[column].astype('int64')
    print('---------mid_2-----------')
    df['person_attribute_a']=df['person_attribute_a_1'].apply(lambda x: x+1)*(df['person_attribute_a'].apply(lambda x:1 if x==1 else -1 ))
    df['contents_attribute_j']=df['contents_attribute_j'].apply(lambda x: 0 if x==1 else 10)+df['contents_attribute_j_1']
    print('---------mid_3----------')
    df['contents_attribute_k']-=1
    #df=pd.get_dummies(df,columns=['person_prefer_c','contents_attribute_i','contents_attribute_a',
    #                              'contents_attribute_j','contents_attribute_c','contents_attribute_m','contents_open_dt'])
    df.drop(['person_attribute_a_1','contents_attribute_j_1'],axis=1,inplace=True)
    print("-----------finish------------")
    return df

In [179]:
train_set=preprocessing(createDF(tmp_train))
test_set=preprocessing(createDF(tmp_test))

--------start---------
---------mid_1----------
---------mid_2-----------
---------mid_3----------
-----------finish------------
--------start---------
---------mid_1----------
---------mid_2-----------
---------mid_3----------
-----------finish------------


In [180]:
train_set.keys()

Index(['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_b', 'person_prefer_c', 'person_prefer_e',
       'contents_attribute_i', 'contents_attribute_a', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_m',
       'contents_attribute_e', 'person_rn', 'contents_open_dt', 'target',
       'person_prefer_d_1_n', 'person_prefer_d_1_s', 'person_prefer_d_1_m',
       'person_prefer_d_1_l', 'person_prefer_d_2_n', 'person_prefer_d_2_s',
       'person_prefer_d_2_m', 'person_prefer_d_2_l', 'person_prefer_d_3_n',
       'person_prefer_d_3_s', 'person_prefer_d_3_m', 'person_prefer_d_3_l',
       'person_prefer_h_1_m', 'person_prefer_h_1_l', 'person_prefer_h_2_m',
       'person_prefer_h_2_l', 'person_prefer_h_3_m', 'person_prefer_h_3_l',
       'contents_attribute_l_n', 'contents_attribute_l_s',
       'contents_attribute_l_m', 'contents_attribute_l

In [181]:
test_set

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_b,person_prefer_c,person_prefer_e,...,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,contents_attribute_h_m,contents_attribute_h_l
id,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,1,1,2,2,1,4,...,1146,1128,1021,2010,836,831,824,744,528,250
1,0,0,0,1,0,0,-1,2,2,2,...,1610,1606,1605,2016,276,274,274,216,528,250
2,1,0,0,1,1,1,-4,2,1,0,...,1812,1811,1810,2020,490,490,482,482,453,169
3,1,0,0,1,1,1,3,2,5,4,...,101,100,99,2006,145,123,56,1,453,169
4,1,0,0,1,0,0,7,4,5,5,...,984,980,954,2009,1097,1094,1093,926,453,169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,1,1,1,0,0,0,-1,4,1,4,...,759,759,759,2006,145,123,56,1,396,85
46400,1,0,0,1,0,0,-1,4,1,4,...,759,759,759,2006,145,123,56,1,396,85
46401,1,1,0,1,1,1,-1,3,1,5,...,759,759,759,2006,145,123,56,1,547,277


In [182]:
key_list=list(train_set.keys())
target_idx=key_list.index('target')
y_train=train_set['target']
x_train=train_set[key_list[:target_idx]+key_list[target_idx+1:]]

In [183]:
y_train

id
0         1
1         0
2         0
3         0
4         0
         ..
501946    1
501947    1
501948    1
501949    1
501950    1
Name: target, Length: 501951, dtype: int64

In [184]:
x_train

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_b,person_prefer_c,person_prefer_e,...,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,contents_attribute_h_m,contents_attribute_h_l
id,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,0,0,0,5,3,5,8,...,1607,1606,1605,2016,275,274,274,216,422,94
1,0,0,0,1,1,0,4,4,1,4,...,1607,1606,1605,2016,275,274,274,216,417,94
2,0,0,0,1,0,0,-1,3,5,3,...,1599,1595,1572,2016,92,91,56,1,363,48
3,0,0,0,1,0,0,-1,2,5,3,...,1607,1606,1605,2016,275,274,274,216,381,71
4,1,1,1,0,0,0,4,4,5,4,...,1607,1606,1605,2016,275,274,274,216,381,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,0,0,0,1,0,0,2,2,2,3,...,352,347,346,2006,145,123,56,1,373,58
501947,1,1,0,1,0,0,7,2,1,4,...,155,138,99,2006,114,109,56,1,425,94
501948,1,1,1,1,0,0,8,4,1,4,...,437,417,407,2006,145,123,56,1,373,58


In [185]:
#로지스틱 회귀
#model=LogisticRegression()
#model.fit(x_train,y_train)

from sklearn.model_selection import GridSearchCV
rf_test=RandomForestClassifier(random_state=25)
parameters={'n_estimators':[i for i in range(50,100,5)],
           'max_depth':[i for i in range(15,30,3)]}
grid_rf=GridSearchCV(rf_test,
                    param_grid = parameters,
                    cv=5,
                    n_jobs=-1
                    )
grid_rf.fit(x_train,y_train)
result=pd.DataFrame(grid_rf.cv_results_['params'])
result['mean_test_score']=grid_rf.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score',ascending=False)

#랜덤포레스트
model=RandomForestClassifier(random_state=25,n_estimators=50,max_depth=20)
model.fit(x_train,y_train)

#인공신경망
model=tf.keras.Sequential()

model.add(layers.Dense(72,input_shape=(72,)))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(36))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='nadam',
             metrics=['acc'])

hist=model.fit(x_train,y_train,
              epochs=500,
              batch_size=128,
              validation_split=0.2,
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=250)],
              verbose=1)

In [186]:
#캣부스트
#cat_features=x_train.columns[x_train.nunique()>2].to_list()
is_holdout=False
cv=KFold(n_splits=5,
        shuffle=True)
cat_features=x_train.columns.to_list()
scores=[]
models=[]
for train_idx,val_idx in cv.split(x_train):
    print('='*30)
    preds=[]
    
    model=CatBoostClassifier(iterations=3000,
                             task_type='GPU',
                             eval_metric='F1',
                             cat_features=cat_features,
                             one_hot_max_size=4)
    
    model.fit(x_train.iloc[train_idx],y_train[train_idx],
              eval_set=[(x_train.iloc[val_idx],y_train[val_idx])],
              early_stopping_rounds=300,
              verbose=100)
    
    models.append(model)
    scores.append(model.get_best_score()['validation']['F1'])
    if is_holdout:
        break
print(scores)
print(np.mean(scores))

Learning rate set to 0.027144
0:	learn: 0.6307863	test: 0.6385348	best: 0.6385348 (0)	total: 1.4s	remaining: 1h 9m 58s
100:	learn: 0.6409484	test: 0.6507022	best: 0.6510000 (99)	total: 1m 42s	remaining: 48m 53s
200:	learn: 0.6498872	test: 0.6633416	best: 0.6633416 (200)	total: 3m 23s	remaining: 47m 8s
300:	learn: 0.6561838	test: 0.6732714	best: 0.6732714 (300)	total: 5m 5s	remaining: 45m 37s
400:	learn: 0.6602954	test: 0.6785591	best: 0.6785591 (400)	total: 6m 47s	remaining: 43m 58s
500:	learn: 0.6637423	test: 0.6802491	best: 0.6802615 (499)	total: 8m 29s	remaining: 42m 19s
600:	learn: 0.6664717	test: 0.6818323	best: 0.6818323 (600)	total: 10m 11s	remaining: 40m 41s
700:	learn: 0.6687166	test: 0.6822131	best: 0.6827866 (627)	total: 11m 52s	remaining: 38m 55s
800:	learn: 0.6701921	test: 0.6825183	best: 0.6827866 (627)	total: 13m 32s	remaining: 37m 11s
900:	learn: 0.6712062	test: 0.6828493	best: 0.6830618 (894)	total: 15m 9s	remaining: 35m 18s
1000:	learn: 0.6723937	test: 0.6828077	best:

In [193]:
threshold=0.35
pred_list=[]
scores_2=[]
for i,(train_idx,val_idx) in enumerate(cv.split(x_train)):
    pred=models[i].predict_proba(x_train.iloc[val_idx])[:,1]
    pred=np.where(pred>=threshold,1,0)
    score=f1_score(y_train[val_idx],pred)
    scores_2.append(score)
    pred=models[i].predict_proba(test_set)[:,1]
    pred_list.append(pred)
print(scores_2)
print(np.mean(scores_2))

[0.7827051354175039, 0.7837374993941547, 0.7859280829973799, 0.7891539705089893, 0.7883131022603551]
0.7859675581156766


In [194]:
#preds=model.predict(test_set) # catboost 말고
preds=np.mean(pred_list,axis=0) # catboost
preds=np.where(preds>=threshold,1,0) # catboost
submission=pd.read_csv('sample_submission.csv')
submission['target']=preds.round()
submission.to_csv('D:/Users/user/Desktop/submission.csv',index=False)

In [131]:
len(cat_features)

40